Compare different neighbourhoods to choose the best one to live. The name of the capstone project is the Battle of neighbourhood. This notebook is used for this project.

In [270]:
import pandas
import numpy
from sklearn.cluster import KMeans

In [199]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [200]:
import pandas as pd
import wikipedia as wp

In [201]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html,header=0)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [202]:
df=df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [203]:
df['Neighbourhood']=df.groupby(["Postcode", "Borough"])['Neighbourhood'].transform(lambda x: ','.join(x))

In [204]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"


In [205]:
for i in range(0,df.shape[0]):
    if df.iloc[i]['Neighbourhood']== 'Not assigned':
        df.iloc[i]['Neighbourhood']=df.iloc[i]['Borough']

In [206]:
N=df[df.Neighbourhood == 'Not assigned']
N.head()

,Postcode,Borough,Neighbourhood


I got the data from Wiki using the wikipedia library for Python, that is easier to use than others, after i made some data cleaning to get the final dataset
and checked if there were no Neighbourhood with 'Not assigned' value

In [218]:
df=df.reset_index()
df=df.drop_duplicates()
df.shape

(103, 5)

In [208]:
Geo=pd.read_csv('/Users/it043169/Downloads/Geospatial_Coordinates.csv')
Geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [209]:
Geo=Geo.rename(columns={'Postal Code': 'Postcode'})

In [210]:
df=df.join(Geo.set_index('Postcode'),on='Postcode')
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
5,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
6,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763


In [211]:
import folium

In [212]:
df['Latitude'].mean()

43.69787599245281

And now let's go with the analysis...

In [213]:
map_Toronto = folium.Map(location=[df['Latitude'].mean(),df['Longitude'].mean()], zoom_start=11)

In [186]:
for i in range(0,df.shape[0]):
    neighborhood=df['Neighbourhood'].iloc[i]
    borough=df['Borough'].iloc[i]
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker(
            [df['Latitude'].iloc[i], df['Longitude'].iloc[i]],
            radius=5,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Toronto)
map_Toronto

In [187]:
map_Toronto

Now we use the account of Foursquare to get some info about Toronto

In [188]:
CLIENT_ID = 'JE4WFYGWP3SGO20CU1TXK2ORMWK0SLEYC35EVRAY5NDGFCUU' # your Foursquare ID
CLIENT_SECRET = 'GQTY3OMPZZIUWXMM5FSS5MJMYAX02Z3Q322X1X5PSSYDHZLF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JE4WFYGWP3SGO20CU1TXK2ORMWK0SLEYC35EVRAY5NDGFCUU
CLIENT_SECRET:GQTY3OMPZZIUWXMM5FSS5MJMYAX02Z3Q322X1X5PSSYDHZLF


In [222]:
df=df.reset_index()

In [237]:
df.loc[0,'Latitude']

43.7532586

In [228]:
df=df.drop(['index'],axis=1)

In [230]:
Torontodataset=df

In [244]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [245]:
neighborhood_latitude = df['Latitude'].iloc[0] # neighborhood latitude value
neighborhood_longitude =  df['Longitude'].iloc[0]  # neighborhood longitude value

neighborhood_name = df['Neighbourhood'].iloc[0] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [253]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])
            
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [285]:
LIMIT=110
toronto_venues= getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

In [286]:
toronto_venues.to_csv('toronto_venues')

<b>The following dataset shows how are distribuited the Venues in the Neighborhood of Toronto<b>

In [287]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,5,5,5,5,5,5
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",11,11,11,11,11,11
"Alderwood,Long Branch",11,11,11,11,11,11
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",26,26,26,26,26,26
Berczy Park,55,55,55,55,55,55


In [288]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


In [289]:
# one hot encoding
manhattan_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] =toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [290]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.0,0.010000
1,Agincourt,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwood,Long Branch",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,"Bathurst Manor,Downsview North,Wilson Heights",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,0.0,0.000000
6,Bayview Village,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Bedford Park,Lawrence Manor East",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Berczy Park,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"Birch Cliff,Cliffside West",0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [291]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2           Steakhouse  0.04
3  American Restaurant  0.04
4                  Gym  0.03


----Agincourt----
                venue  freq
0      Sandwich Place   0.2
1              Lounge   0.2
2      Breakfast Spot   0.2
3      Clothing Store   0.2
4  Chinese Restaurant   0.2


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                             venue  freq
0                             Park  0.67
1                       Playground  0.33
2                      Yoga Studio  0.00
3                    Metro Station  0.00
4  Molecular Gastronomy Restaurant  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.18
1           Pizza Place  0.09
2  Fast Food Restaurant  0.09
3              Pharmacy  0.09
4            Beer Store  0.09


--

                  venue  freq
0    Athletics & Sports   0.2
1        Discount Store   0.2
2  Gym / Fitness Center   0.2
3          Liquor Store   0.2
4         Grocery Store   0.2


----Downsview West----
                             venue  freq
0                    Grocery Store  0.50
1                    Shopping Mall  0.25
2                             Bank  0.25
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----East Birchmount Park,Ionview,Kennedy Park----
              venue  freq
0    Discount Store  0.25
1       Coffee Shop  0.12
2       Bus Station  0.12
3  Department Store  0.12
4     Train Station  0.12


----East Toronto----
                venue  freq
0         Coffee Shop  0.33
1   Convenience Store  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Mexican Restaurant  0.00


----Emery,Humberlea----
                             venue  freq
0                   Baseball Field   1.0
1                      Yoga Studio   0.0
2

              venue  freq
0       Coffee Shop  0.24
1      Burger Joint  0.05
2               Gym  0.05
3  Sushi Restaurant  0.05
4             Diner  0.05


----Rosedale----
           venue  freq
0           Park  0.50
1     Playground  0.25
2          Trail  0.25
3    Yoga Studio  0.00
4  Metro Station  0.00


----Roselawn----
                             venue  freq
0                           Garden   0.5
1                             Pool   0.5
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Rouge,Malvern----
                             venue  freq
0             Fast Food Restaurant   1.0
1                    Metro Station   0.0
2  Molecular Gastronomy Restaurant   0.0
3       Modern European Restaurant   0.0
4                Mobile Phone Shop   0.0


----Runnymede,Swansea----
                venue  freq
0         Pizza Place  0.09
1         Coffee Shop  0.09
2                Café  0.09
3  Italian R

In [292]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [293]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Steakhouse,Restaurant,Bar,Gym,Hotel,Cosmetics Shop,Thai Restaurant
1,Agincourt,Lounge,Chinese Restaurant,Sandwich Place,Clothing Store,Breakfast Spot,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Women's Store,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Beer Store,Fried Chicken Joint,Japanese Restaurant,Fast Food Restaurant,Discount Store,Coffee Shop,Pizza Place,Sandwich Place,Pharmacy
4,"Alderwood,Long Branch",Pizza Place,Gym,Coffee Shop,Skating Rink,Dance Studio,Pharmacy,Bank,Pub,Sandwich Place,Pool
5,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Grocery Store,Frozen Yogurt Shop,Sandwich Place,Bridal Shop,Diner,Bank,Restaurant,Deli / Bodega,Sushi Restaurant
6,Bayview Village,Bank,Café,Japanese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Women's Store
7,"Bedford Park,Lawrence Manor East",Italian Restaurant,Fast Food Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Liquor Store,Sandwich Place,Comfort Food Restaurant,Juice Bar,Restaurant
8,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Café,Steakhouse,Restaurant,Bakery,Pub,Seafood Restaurant,Cheese Shop
9,"Birch Cliff,Cliffside West",General Entertainment,College Stadium,Café,Skating Rink,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [294]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [297]:
manhattan_grouped['cluster']=kmeans.labels_

In [298]:
cluster=manhattan_grouped[['Neighborhood','cluster']]

In [301]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [305]:
cluster=cluster.rename(columns={'Neighborhood': 'Neighbourhood'})
dataset_cluster=df.join(cluster.set_index('Neighbourhood'),on='Neighbourhood')

In [306]:
dataset_cluster.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,cluster
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0.0
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0


In [308]:
neighborhoods_venues_sorted=neighborhoods_venues_sorted.rename(columns={'Neighborhood': 'Neighbourhood'})
dataset_cluster.join(neighborhoods_venues_sorted.set_index('Neighbourhood'),on='Neighbourhood')

,Postcode,Borough,Neighbourhood,Latitude,Longitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Design Studio
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Mexican Restaurant,Restaurant,Breakfast Spot,Farmers Market
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0.0,Clothing Store,Furniture / Home Store,Coffee Shop,Event Space,Boutique,Shoe Store,Vietnamese Restaurant,Accessories Store,Miscellaneous Shop,Electronics Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Diner,Burger Joint,Gym,Japanese Restaurant,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Smoothie Shop,Seafood Restaurant
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Design Studio,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
7,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Baseball Field,Gym / Fitness Center,Japanese Restaurant,Café,Caribbean Restaurant,Pool,Donut Shop,Diner,Discount Store,Dog Run
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0.0,Fast Food Restaurant,Pizza Place,Pet Store,Athletics & Sports,Gastropub,Intersection,Pharmacy,Café,Rock Climbing Spot,Bank
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Diner,Tea Room,Sandwich Place


In [309]:
cluster=dataset_cluster.join(neighborhoods_venues_sorted.set_index('Neighbourhood'),on='Neighbourhood')

In [367]:
cluster.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Fast Food Restaurant,Park,Food & Drink Shop,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Design Studio
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Mexican Restaurant,Restaurant,Breakfast Spot,Farmers Market
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Coffee Shop,Event Space,Boutique,Shoe Store,Vietnamese Restaurant,Accessories Store,Miscellaneous Shop,Electronics Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Diner,Burger Joint,Gym,Japanese Restaurant,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Smoothie Shop,Seafood Restaurant


In [344]:
for i in range(0,cluster.shape[0]-1):
    if  cluster['cluster'][i] >=0:
        setting=1
    else:
        cluster['cluster'][i]=6
 

0
1
2
3
4


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
96
97
98
99
100
101


In [347]:
cluster['cluster']=cluster['cluster'].astype(int)

In [361]:
cluster1=cluster.query('cluster==1')
cluster2=cluster.query('cluster==2')
cluster3=cluster.query('cluster==3')
cluster4=cluster.query('cluster==4')
cluster0=cluster.query('cluster==0')
cluster6=cluster.query('cluster==6')

In [351]:
map_Toronto = folium.Map(location=[cluster['Latitude'].mean(),cluster['Longitude'].mean()], zoom_start=11)

<b> The Rappresentation of Toronto according the clusterization <b>

In [365]:
for i in range(0,cluster1.shape[0]):
    folium.CircleMarker(
            [cluster1['Latitude'].iloc[i], cluster1['Longitude'].iloc[i]],
            radius=5,
            color='blue',
            parse_html=False).add_to(map_Toronto)

for i in range(0,cluster2.shape[0]):
    folium.CircleMarker(
            [cluster2['Latitude'].iloc[i], cluster2['Longitude'].iloc[i]],
            radius=5,
            color='red',
            parse_html=False).add_to(map_Toronto)

for i in range(0,cluster3.shape[0]):
    folium.CircleMarker(
            [cluster3['Latitude'].iloc[i], cluster3['Longitude'].iloc[i]],
            radius=5,
            color='orange',
            parse_html=False).add_to(map_Toronto)
    
for i in range(0,cluster4.shape[0]):
    folium.CircleMarker(
            [cluster4['Latitude'].iloc[i], cluster4['Longitude'].iloc[i]],
            radius=5,
            color='yellow',
            parse_html=False).add_to(map_Toronto)
    
for i in range(0,cluster0.shape[0]):
    folium.CircleMarker(
            [cluster0['Latitude'].iloc[i], cluster0['Longitude'].iloc[i]],
            radius=5,
            color='green',
            parse_html=False).add_to(map_Toronto)

for i in range(0,cluster6.shape[0]):
    folium.CircleMarker(
            [cluster6['Latitude'].iloc[i], cluster6['Longitude'].iloc[i]],
            radius=5,
            color='black',
            parse_html=False).add_to(map_Toronto)
    
map_Toronto

In [366]:
cluster0#The most common type of neighboor...????

,Postcode,Borough,Neighbourhood,Latitude,Longitude,cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Hockey Arena,Coffee Shop,Portuguese Restaurant,Intersection,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Design Studio
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Café,Theater,Mexican Restaurant,Restaurant,Breakfast Spot,Farmers Market
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0,Clothing Store,Furniture / Home Store,Coffee Shop,Event Space,Boutique,Shoe Store,Vietnamese Restaurant,Accessories Store,Miscellaneous Shop,Electronics Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0,Coffee Shop,Diner,Burger Joint,Gym,Japanese Restaurant,Sushi Restaurant,Italian Restaurant,Fast Food Restaurant,Smoothie Shop,Seafood Restaurant
7,M3B,North York,Don Mills North,43.745906,-79.352188,0,Baseball Field,Gym / Fitness Center,Japanese Restaurant,Café,Caribbean Restaurant,Pool,Donut Shop,Diner,Discount Store,Dog Run
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937,0,Fast Food Restaurant,Pizza Place,Pet Store,Athletics & Sports,Gastropub,Intersection,Pharmacy,Café,Rock Climbing Spot,Bank
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Italian Restaurant,Diner,Tea Room,Sandwich Place
10,M6B,North York,Glencairn,43.709577,-79.445073,0,Pizza Place,Japanese Restaurant,Metro Station,Pub,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Doner Restaurant
13,M3C,North York,"Flemingdon Park,Don Mills South",43.725900,-79.340923,0,Asian Restaurant,Gym,Coffee Shop,Beer Store,Grocery Store,Clothing Store,Chinese Restaurant,Dim Sum Restaurant,Restaurant,Sandwich Place
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Hotel,Clothing Store,Cosmetics Shop,Bakery,Cocktail Bar,Gastropub,Creperie
